In [8]:

from sklearn.model_selection import train_test_split as TTS
from sklearn.metrics import mean_squared_error as MSE, r2_score
import pickle #模型封装
import xgboost as xgb
import pandas as pd # 基础的运算库，必备，下同
import numpy as np
from sklearn.preprocessing import StandardScaler # 数据标准化 均值为0，方差为1的正态分布
from sklearn.impute import SimpleImputer # 数据去零补缺

from sklearn.model_selection import KFold, cross_val_score as CVS
import matplotlib.pyplot as plt 
from time import time
import datetime
from xgboost import XGBRegressor as XGBR
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LinearR

In [9]:
## 训练数据导入

Data = pd.read_csv(r"F:\lastpaper\data\L8_20191105_docCDOM_2.csv"
                   , index_col = 0
                  )

X = Data.iloc[:,2:9]
X.rename(columns={'b1_L8_9111':'0'
                 ,"b2_L8_9111":'1'
                 ,"b3_L8_9111":'2'
                 ,"b4_L8_9111":'3'
                 ,"b5_L8_9111":'4'
                 ,"b6_L8_9111":'5'
                 ,"b7_L8_9111":'6'
                 },inplace=True)
#     X = np.array(X)
#     for col in range(X.shape[1]):
#         median=np.median(X[:,col])
#         for row in range(X.shape[0]):
#             if X[row,col] <= 0:
#                 X[row,col] = median
#     scaler = StandardScaler() # 标准化
#     X = scaler.fit_transform(X)

Y = Data.iloc[:,0]
Xtrain,Xtest,Ytrain,Ytest = TTS(X,Y,test_size=0.3,random_state=40)
X

#Y

C:\Users\RSof4\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,0,1,2,3,4,5,6
Name,,,,,,,
DW071R,624.0,503.0,692.0,550.0,1352.0,615.0,310.0
DW072R,593.0,463.0,670.0,544.0,728.0,243.0,122.0
DW073R,601.0,478.0,690.0,535.0,1157.0,309.0,142.0
DW074R,562.0,432.0,621.0,485.0,1435.0,459.0,198.0
DW075R,714.0,545.0,696.0,529.0,2221.0,622.0,247.0
...,...,...,...,...,...,...,...
DW294R,651.0,563.0,825.0,740.0,261.0,128.0,83.0
DW295R,655.0,554.0,805.0,766.0,321.0,128.0,84.0
DW296R,694.0,605.0,839.0,768.0,290.0,129.0,84.0


In [10]:
## 模型封装


import pickle
import xgboost as xgb

dtrain = xgb.DMatrix(Xtrain,Ytrain)

#设定参数，对模型进行训练 
param = {'silent':True
          ,'obj':'reg:linear'
          ,"subsample":1
          ,"max_depth":6
          ,"eta":0.15
          ,"gamma":0.9
          ,"lambda":1
          ,"alpha":0.255
          ,"colsample_bytree":1
          ,"colsample_bylevel":1
          ,"colsample_bynode":1
          ,"nfold":5}
num_round = 30

bst = xgb.train(param, dtrain, num_round)

#保存模型
pickle.dump(bst, open("xgboost_cdom.dat","wb"))
#注意，open中我们往往使用w或者r作为读取的模式，但其实w与r只能用于文本文件，当我们希望导入的不是文本文件，而
#是模型本身的时候，我们使用"wb"和"rb"作为读取的模式。其中wb表示以二进制写入，rb表示以二进制读入
#loaded_model = pickle.load(open("xgboost_cdom.dat", "rb"))

[20:00:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { nfold, obj, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [5]:
## 模型评估调用

#注意，如果我们保存的模型是xgboost库中建立的模型，则导入的数据类型也必须是xgboost库中的数据类型
dtrain = xgb.DMatrix(Xtrain,Ytrain)
dtest = xgb.DMatrix(Xtest,Ytest)
#导入模型
loaded_model = pickle.load(open("xgboost_cdom.dat", "rb"))
print("Loaded model from: xgboost_cdom.dat")


Loaded model from: xgboost_cdom.dat


In [17]:
# 模型效率
from time import time
import datetime
import xgboost as xgb
time0 = time()
xgb_model=xgb.train(param, dtrain, num_round)

time= datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f") #计算时间戳并转换为时分秒

#做预测
yprods= xgb_model.predict(dtrain)
ypreds = xgb_model.predict(dtest)

from sklearn.metrics import mean_squared_error as MSE,r2_score
test_mse=MSE(Ytest,ypreds)
test_r2 = r2_score(Ytest,ypreds)
fit_r2 = r2_score(Ytrain,yprods)
fit_mse = MSE(Ytrain,yprods)
prams= [['Fittime','FitMSE','FitR2','TestMSE','TestR2'],
        [time,fit_mse,fit_r2,test_mse,test_r2]]
pd.DataFrame(prams)

[20:17:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { nfold, obj, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




,0,1,2,3,4
0,Fittime,FitMSE,FitR2,TestMSE,TestR2
1,00:00:031915,0.170069,0.838784,0.238881,0.733752


In [18]:
# coding=UTF-8
import xlrd
import xlwt

def write_excel_xls(path, sheet_name, value):
    workbook = xlwt.Workbook()  # 新建一个工作簿
    for k in range(len(value)):
        index = len(value[k])  # 获取需要写入数据的行数
        data = value[k]
        name = sheet_name[k]
        sheet = workbook.add_sheet(name)  # 在工作簿中新建一个表格
        for i in range(0, index):
            for j in range(0, len(data[i])):
                sheet.write(i, j, str(data[i][j]))  # 像表格中写入数据（对应的行和列）
    workbook.save(path)  # 保存工作簿
    print("xls格式表格写入数据成功！")

book_name_xls = 'XGBoost.xls'
 
sheet_name_xls = ['prams','testdata','traindata']
 
value = [[prams],[Ytest,ypreds],[Ytrain,yprods]]
 
write_excel_xls(book_name_xls,sheet_name_xls, value)

xls格式表格写入数据成功！
